In [1]:
!pip install selenium
!pip install chromedriver-autoinstaller
!pip install pandas
!pip install requests
!pip install bs4

In [2]:
import pandas as pd
import time
import requests
import chromedriver_autoinstaller

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

In [3]:
service = Service()

options = webdriver.ChromeOptions()

driver = webdriver.Chrome(service=service, options=options)

In [4]:
driver.get("https://anmlegis.datalegis.net/action/ActionDatalegis.php?acao=apresentacao&cod_modulo=351&cod_menu=8014")
time.sleep(5)

In [5]:
driver.find_element(By.XPATH,'//*[@id="column-2"]/div[1]/a[4]').click()
#clica em "Títulos Outorgados"

In [6]:
driver.find_element(By.XPATH,'//*[@id="column-2"]/div/div[2]/div[4]/ul/li[6]/a').click()
time.sleep(5)
#clica em "Registro de extração"

In [7]:
driver.find_element(By.XPATH,'//*[@id="column-2"]/div[1]/ul/li[1]/a').click()
time.sleep(5)
#clica em "2024"

In [8]:
last_height2024 = driver.execute_script("return document.body.scrollHeight")

In [9]:
for vez2024 in range (3):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

    new_height2024 = driver.execute_script("return document.body.scrollHeight")

    if new_height2024 == last_height2024 :
        break   
    last_height2024 = new_height2024

In [10]:
soup2024 = BeautifulSoup(driver.page_source, 'html.parser')

In [11]:
tabela2024 = soup2024.find('div', {'class': {'col-lg-11 col-md-11 col-sm-12'}})

In [12]:
titulos2024 = tabela2024.find_all('strong', {})
conteudos2024 = tabela2024.find_all('p')[0:300:2]
datasPublicacao2024 = tabela2024.find_all('p', {'class': 'data-hora'})

In [13]:
lista_titulos2024 = []
lista_conteudos2024 = []
lista_datasPublicacao2024 = []

for titulo2024 in titulos2024:
  lista_titulos2024.append(titulo2024.get_text().strip())

for conteudo2024 in conteudos2024:
  lista_conteudos2024.append(conteudo2024.get_text().strip())

for dataPublicacao2024 in datasPublicacao2024:
  lista_datasPublicacao2024.append(dataPublicacao2024.get_text().strip())

df_extracao2024 = pd.DataFrame([lista_titulos2024, lista_conteudos2024, lista_datasPublicacao2024]).T

In [14]:
driver.back()

In [15]:
driver.find_element(By.XPATH,'//*[@id="column-2"]/div[1]/ul/li[2]/a').click()
time.sleep(5)
#clica em "2023"

In [16]:
last_height2023 = driver.execute_script("return document.body.scrollHeight")

In [17]:
for vez2023 in range (9):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

    new_height2023 = driver.execute_script("return document.body.scrollHeight")

    if new_height2023 == last_height2023 :
        break   
    last_height2023 = new_height2023

In [18]:
soup2023 = BeautifulSoup(driver.page_source, 'html.parser')

In [19]:
tabela2023 = soup2023.find('div', {'class': {'col-lg-11 col-md-11 col-sm-12'}})

In [20]:
titulos2023 = tabela2023.find_all('strong', {})
conteudos2023 = tabela2023.find_all('p')[0:400:2]
datasPublicacao2023 = tabela2023.find_all('p', {'class': 'data-hora'})

In [21]:
lista_titulos2023 = []
lista_conteudos2023 = []
lista_datasPublicacao2023 = []

for titulo2023 in titulos2023:
  lista_titulos2023.append(titulo2023.get_text().strip())

for conteudo2023 in conteudos2023:
  lista_conteudos2023.append(conteudo2023.get_text().strip())

for dataPublicacao2023 in datasPublicacao2023:
  lista_datasPublicacao2023.append(dataPublicacao2023.get_text().strip())

df_extracao2023 = pd.DataFrame([lista_titulos2023, lista_conteudos2023, lista_datasPublicacao2023]).T

In [22]:
driver.back()

In [23]:
driver.find_element(By.XPATH,'//*[@id="column-2"]/div[1]/ul/li[3]/a').click()
time.sleep(5)
#clica em "2022"

In [24]:
last_height2022 = driver.execute_script("return document.body.scrollHeight")

In [25]:
for vez2022 in range (4):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

    new_height2022 = driver.execute_script("return document.body.scrollHeight")

    if new_height2022 == last_height2022 :
        break   
    last_height2022 = new_height2022

In [26]:
soup2022 = BeautifulSoup(driver.page_source, 'html.parser')

In [27]:
tabela2022 = soup2022.find('div', {'class': {'col-lg-11 col-md-11 col-sm-12'}})

In [28]:
titulos2022 = tabela2022.find_all('strong', {})
conteudos2022 = tabela2022.find_all('p')[0:200:2]
datasPublicacao2022 = tabela2022.find_all('p', {'class': 'data-hora'})

In [29]:
lista_titulos2022 = []
lista_conteudos2022 = []
lista_datasPublicacao2022 = []

for titulo2022 in titulos2022:
  lista_titulos2022.append(titulo2022.get_text().strip())

for conteudo2022 in conteudos2022:
  lista_conteudos2022.append(conteudo2022.get_text().strip())

for dataPublicacao2022 in datasPublicacao2022:
  lista_datasPublicacao2022.append(dataPublicacao2022.get_text().strip())

df_extracao2022 = pd.DataFrame([lista_titulos2022, lista_conteudos2022, lista_datasPublicacao2022]).T

In [30]:
driver.back()

In [31]:
driver.find_element(By.XPATH,'//*[@id="column-2"]/div[1]/ul/li[4]/a').click()
time.sleep(5)
#clica em "2021"

In [32]:
last_height2021 = driver.execute_script("return document.body.scrollHeight")

In [33]:
for vez2021 in range (4):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

    new_height2021 = driver.execute_script("return document.body.scrollHeight")

    if new_height2021 == last_height2021 :
        break   
    last_height2021 = new_height2021

In [34]:
soup2021 = BeautifulSoup(driver.page_source, 'html.parser')

In [35]:
tabela2021 = soup2021.find('div', {'class': {'col-lg-11 col-md-11 col-sm-12'}})

In [36]:
titulos2021 = tabela2021.find_all('strong', {})
conteudos2021 = tabela2021.find_all('p')[0:200:2]
datasPublicacao2021 = tabela2021.find_all('p', {'class': 'data-hora'})

In [37]:
lista_titulos2021 = []
lista_conteudos2021 = []
lista_datasPublicacao2021 = []

for titulo2021 in titulos2021:
  lista_titulos2021.append(titulo2021.get_text().strip())

for conteudo2021 in conteudos2021:
  lista_conteudos2021.append(conteudo2021.get_text().strip())

for dataPublicacao2021 in datasPublicacao2021:
  lista_datasPublicacao2021.append(dataPublicacao2021.get_text().strip())

df_extracao2021 = pd.DataFrame([lista_titulos2021, lista_conteudos2021, lista_datasPublicacao2021]).T

In [38]:
driver.back()

In [39]:
driver.find_element(By.XPATH,'//*[@id="column-2"]/div[1]/ul/li[5]/a').click()
time.sleep(5)
#clica em "2020"

In [40]:
last_height2020 = driver.execute_script("return document.body.scrollHeight")

In [41]:
for vez2020 in range (7):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

    new_height2020 = driver.execute_script("return document.body.scrollHeight")

    if new_height2020 == last_height2020 :
        break   
    last_height2020 = new_height2020

In [42]:
soup2020 = BeautifulSoup(driver.page_source, 'html.parser')

In [43]:
tabela2020 = soup2020.find('div', {'class': {'col-lg-11 col-md-11 col-sm-12'}})

In [44]:
titulos2020 = tabela2020.find_all('strong', {})
conteudos2020 = tabela2020.find_all('p')[0:300:2]
datasPublicacao2020 = tabela2020.find_all('p', {'class': 'data-hora'})

In [45]:
lista_titulos2020 = []
lista_conteudos2020 = []
lista_datasPublicacao2020 = []

for titulo2020 in titulos2020:
  lista_titulos2020.append(titulo2020.get_text().strip())

for conteudo2020 in conteudos2020:
  lista_conteudos2020.append(conteudo2020.get_text().strip())

for dataPublicacao2020 in datasPublicacao2020:
  lista_datasPublicacao2020.append(dataPublicacao2020.get_text().strip())

df_extracao2020 = pd.DataFrame([lista_titulos2020, lista_conteudos2020, lista_datasPublicacao2020]).T

In [46]:
registroExtracao = pd.concat([df_extracao2020, df_extracao2021, df_extracao2022, df_extracao2023, df_extracao2024])

In [47]:
registroExtracao

,0,1,2
0,REGISTRO DE EXTRAÇÃO Nº 73/2020,"Expede a DECLARAÇÃO DE REGISTRO DE EXTRAÇÃO, p...",31/12/2020 | 09:26:08
1,REGISTRO DE EXTRAÇÃO Nº 72/2020,"Expede a DECLARAÇÃO DE REGISTRO DE EXTRAÇÃO, p...",31/12/2020 | 09:10:13
2,REGISTRO DE EXTRAÇÃO Nº 173/2020,"Expede a DECLARAÇÃO DE REGISTRO DE EXTRAÇÃO, p...",30/12/2020 | 10:51:46
3,REGISTRO DE EXTRAÇÃO Nº 172/2020,"Expedir a DECLARAÇÃO DE REGISTRO DE EXTRAÇÃO, ...",30/12/2020 | 10:48:50
4,REGISTRO DE EXTRAÇÃO Nº 170/2020,"Expede a DECLARAÇÃO DE REGISTRO DE EXTRAÇÃO, p...",30/12/2020 | 12:24:23
...,...,...,...
46,REGISTRO DE EXTRAÇÃO Nº 4/2024,"Expede a DECLARAÇÃO DE REGISTRO DE EXTRAÇÃO, p...",24/01/2024 | 08:25:00
47,REGISTRO DE EXTRAÇÃO Nº 3/2024,"Expede a DECLARAÇÃO DE REGISTRO DE EXTRAÇÃO, p...",24/01/2024 | 08:20:02
48,REGISTRO DE EXTRAÇÃO Nº 2/2024,"Expede a DECLARAÇÃO DE REGISTRO DE EXTRAÇÃO, p...",24/01/2024 | 08:17:56
49,REGISTRO DE EXTRAÇÃO Nº 1/2024,"Expede a DECLARAÇÃO DE REGISTRO DE EXTRAÇÃO, p...",24/01/2024 | 08:12:07
